**Importing** **Libraries** 

In [1]:
import pandas as pd
import numpy as np

**Importing Dataset**

In [2]:
dataset=pd.read_csv('spam.csv',encoding='latin-1')
x=dataset.iloc[:,[0]]
y=dataset.iloc[:,[1]]

In [3]:
print(x)

        v1
0      ham
1      ham
2     spam
3      ham
4      ham
...    ...
5567  spam
5568   ham
5569   ham
5570   ham
5571   ham

[5572 rows x 1 columns]


In [4]:
print(y)

                                                     v2
0     Go until jurong point, crazy.. Available only ...
1                         Ok lar... Joking wif u oni...
2     Free entry in 2 a wkly comp to win FA Cup fina...
3     U dun say so early hor... U c already then say...
4     Nah I don't think he goes to usf, he lives aro...
...                                                 ...
5567  This is the 2nd time we have tried 2 contact u...
5568              Will Ì_ b going to esplanade fr home?
5569  Pity, * was in mood for that. So...any other s...
5570  The guy did some bitching but I acted like i'd...
5571                         Rofl. Its true to its name

[5572 rows x 1 columns]


**Encoding into classification(ham(0) or spam(1) )**

In [5]:
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
x=lb.fit_transform(x)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
print(x)

[0 0 1 ... 0 0 0]


**Stemming**

In [7]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
reply=[]
for i in range(0,5572):
  text=re.sub('[^a-zA-Z]',' ',dataset['v2'][i])
  text=text.lower()
  text=text.split()
  
  ps=PorterStemmer()
  text=[ps.stem(word) for word in text if not word  in set(stopwords.words('english'))]
  text=' '.join(text)
  reply.append(text)

**Converting the collection of texts to a matrix of token counts**

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(6200)
x=cv.fit_transform(reply).toarray()
y=dataset.iloc[:,[0]].values

**Splitting dataset into train_set and test_set**

In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

**Training the model**

In [12]:
from sklearn.naive_bayes import GaussianNB
classifier=GaussianNB()
classifier.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB(priors=None, var_smoothing=1e-09)

**Predicting the Test set results**

In [13]:
y_pred=classifier.predict(x_test)
print(np.concatenate( ( y_test.reshape(len(y_test),1),y_pred.reshape(len(y_pred),1) ) ,1))

[['ham' 'ham']
 ['ham' 'ham']
 ['ham' 'ham']
 ...
 ['spam' 'spam']
 ['spam' 'spam']
 ['ham' 'ham']]


In [14]:
from sklearn.metrics import accuracy_score,confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)


[[860 101]
 [ 25 129]]


0.8869955156950673

**Predicting a (new)text**

In [15]:
new_review = 'The IRS is Trying to Contact You'
new_review = re.sub('[^a-zA-Z]', ' ', new_review)
new_review = new_review.lower()
new_review = new_review.split()

ps = PorterStemmer()
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')
new_review = [ps.stem(word) for word in new_review if not word in set(all_stopwords)]
new_review = ' '.join(new_review)
new_corpus = [new_review]
new_X_test = cv.transform(new_corpus).toarray()
new_y_pred = classifier.predict(new_X_test)
print(new_y_pred)

['spam']


In [16]:
# https://www.kaggle.com/uciml/zsms-spam-collection -- Dataset